In [59]:
import os
import glob
import ollama
import chromadb as CM
from chromadb import Settings, EmbeddingFunction, Embeddings

## Create embeddings

In [60]:
ollama_client = ollama.Client(
    host="https://b068-34-73-226-109.ngrok-free.app/",
    headers={"Header": "application/json"},
)
import numpy as np


class MyEmbeddingFunction(EmbeddingFunction):
    def __call__(self, input: str) -> Embeddings:
        embeddings = ollama_client.embed(
            model="mxbai-embed-large",
            input=input,
        )
        return np.array(embeddings.embeddings)

## Create vector db


In [61]:
client = CM.PersistentClient(
    path="../data/vector",
    settings=Settings(
        allow_reset=True,
    ),
)
client.reset()


collection = client.create_collection(
    "VNLaws", embedding_function=MyEmbeddingFunction()
)

In [ ]:
chunk_folder = "../data/chunk/TaiChinh"


# Read chunks and push embeddings into vector DB
for chunk_file in glob.glob(os.path.join(chunk_folder, "**.txt")):
    with open(chunk_file, "r", encoding="utf-8") as f:
        chunk_text = f.read()
        print(chunk_file)

        # push to vector DB
        collection.add(
            documents=[chunk_text],
            ids=[str(hash(chunk_text))],
        )

## Query


In [ ]:
collection.query(
    query_texts=["Cổng thông tin điện tử"],
    n_results=1,
)
# https://docs.trychroma.com/docs/querying-collections/query-and-get